# Probabilistic Models

In [1]:
import os
import re
import pickle
import time
import numpy as np
from IndexerCACM import *
from IndexerQuery import *
from RelevantParser import *
from Query import *
from IRmodel import *
from Vector import *
from IRList import *
from EvalMeasure import *
from EvalIRModel import *
from copy import *

### Get Indexers

In [2]:
# Processed collections 
collectionPath = 'data/cacm/cacm.txt'
collectionPath2 = 'data/cisi/cisi.txt'
queriesPath = 'data/cacm/cacm.qry'
relevantPath = 'data/cacm/cacm.rel'

In [3]:
indexer = IndexerCACM(collectionPath, ParserCACM())

# If Index and Inv Index aren't already builded
#indexer.createRepIndex()
#indexer.createRepInvIndex()
#indexer.createRepInvFromAll()

queriesIndexer = IndexerQuery(queriesPath, ParserCACM())

# If Index isn't already builded
#queriesIndexer.createRepIndex()

relevantIndexer = Indexer(relevantPath, RelevantParser())

# If Index and Inv Index aren't already builded
#relevantIndexer.createIndex()

In [9]:
q = query(1, queriesIndexer, relevantIndexer)
q2 = query(2, queriesIndexer, relevantIndexer)
q4 = query(4, queriesIndexer, relevantIndexer)
print(q.text, q.el)

(' \n  \n  \n   What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers?', {'What': 1.0, 'comput': 1.0, 'IBM': 1.0, 'deal': 1.0, 'Share': 1.0, 'System': 1.0, 'articl': 1.0, 'exist': 1.0, 'operat': 1.0, 'Time': 1.0, 'TSS': 1.0, -1: 11.0})


## Language Model

$P_M(s) = \prod_{i=1}^n p_M(s_i)$

where $P_M(s)$ probability to observe sequence of words $s$ with language model $M$.

We want to use this measure to compare documents with a query so we can be satisfied with something proportional to this (so we can use $\log$).

It leads to this measure :

$f(q,d) = \log P_{M_d}(q) = \sum_{t\in q} tf(t,q)\log p_{M_d}(t)$

where $q$ is a query, $d$ a document, $t$ a term.

And we have $p_{M_d}(t)$ equal to $\frac{tf(t,d)}{L(d)}$

** Sometimes t from q isn't in d ** it leads to,

$\log p_M(t) = -\infty $ cad $\log P_{M_d}(q) = -\infty$ even if other terms from q give a good similarity with $M_d$.

Good method to avoid that is a smoothed prob :

$\log p_M(t) = \log(\lambda p_{M_d}(t)+(1-\lambda)p_{M_c}(t)))$

where $M_C$ is the model language on the whole corpus.

In [40]:
class LanguageModel(IRmodel):
    
    def __init__(self, indexer, lbd=0.5):

        IRmodel.__init__(self, indexer)
        self.lbd = lbd
    
    def probModelDoc(self, id, element):
        
        freq = indexer.getEfFromDoc(id)
        if element in freq:
            return freq[element]/float(freq[-1])
        else:
            return 0
    
    def probModelAll(self, element):
        
        if element in indexer.repInvFromAll:
            return indexer.repInvFromAll[element] \
                /float(indexer.repInvFromAll[-1])
        else:
            return 0
            
    def getScores(self, query):
        
        query = copy(query)
        query.pop(-1)
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        
        dic = {}
        for element in query:
            for doc in self.indexer.getDfFromEl(element):
        for element in query:
            i = 0
            tf = query[element]
            if element in self.indexer.invIndex:
                docFromElement = self.indexer.getDfFromEl(element)
                for id in self.indexer.index:
                    scores[i]['id'] = id
                    if id in docFromElement:
                        scores[i]['score'] += tf * np.log(self.lbd*self.probModelDoc(id, element) \
                        + (1 - self.lbd)*self.probModelAll(element))
                    else:
                        scores[i]['score'] += tf * np.log((1 - self.lbd)*self.probModelAll(element))
                    i += 1
                
        return np.array(scores)
                

In [41]:
indexer.repInvFromAll[-1]
print(indexer.repInvFromAll['I'])
q4.relevants

231.0


[['1749', 0, 0],
 ['1811', 0, 0],
 ['2256', 0, 0],
 ['2371', 0, 0],
 ['2597', 0, 0],
 ['2796', 0, 0],
 ['2912', 0, 0],
 ['3043', 0, 0],
 ['3073', 0, 0],
 ['3082', 0, 0],
 ['3127', 0, 0],
 ['3128', 0, 0]]

In [44]:
Lm = LanguageModel(indexer, lbd=0.5)
scores =  Lm.getRanking(q4.el)
scores[:10]

14.3170311451


array([('3128', -216.44759458713267), ('2939', -226.94479233636926),
       ('1588', -227.57528650371182), ('1135', -227.93761782382936),
       ('2342', -228.1026346065395), ('616', -228.65537437398334),
       ('3101', -229.90185812985624), ('2931', -230.07503273745849),
       ('2470', -230.6887267114721), ('1012', -231.5323262885544)], 
      dtype=[('id', 'S25'), ('score', '<f8')])

## Okapi BM25 Model

$f(d,q) = \sum_{t\in q}idf'(t)\frac{(k_1+1)tf(t,d)}{k_1((1-b)+bL(d)/L_{mean}+tf(t,d))}$ where,

probabilistic $idf'(t) = max(0, \log\frac{N-df(t)+0.5}{df(t)+0.5})$.

$L_{mean}$ is the mean size of documents.

$k_1 \in [1.2, 2.0]$ and $b = 0.75$ are free parameters.

In [67]:
class Okapi(IRmodel):
    
    def __init__(self, indexer, k=1.5, b=0.75):
        
        IRmodel.__init__(self, indexer)
        self.k = k
        self.b = b
        
        self.nDoc = len(indexer.indexFromCol)
        
        
        self.lMean = float(indexer.repInvFromAll[-1])/self.nDoc
        print(self.lMean, self.nDoc)
    
    def idf(self, elements):
        
        result = {}
        
        for element in elements:
            if element in indexer.invIndex:
                df = len(indexer.getDfFromEl(element))-1
                result[element] = \
                max(0, np.log((self.nDoc-df+0.5) / (df+0.5)))
            else:
                result[element] = 0
        
        return result
        
    def getScores(self, query):
        
        query = copy(query)
        query.pop(-1)
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        
        idf = self.idf(query)
        
        for element in query:
            i = 0
            if element in self.indexer.invIndex:
                for id in self.indexer.index:
                    tf = self.indexer.getEfFromDoc(id)
                    docFromElement = self.indexer.getDfFromEl(element)
                    scores[i]['id'] = id
                    if id in docFromElement:
                        scores[i]['score'] \
                        += idf[element] \
                        * ((self.k + 1) * tf[element]) \
                        / (self.k * ((1 - self.b) + self.b * tf[-1] \
                        / self.lMean) + tf[element])
                    i += 1
                
        return np.array(scores)

In [68]:
Lm = Okapi(indexer)
scores =  Lm.getRanking(q4.el)
scores[:10]

(624.8977164605138, 4204)
94.5044410229


array([('3128', 39.88140909621483), ('2342', 33.759712782915656),
       ('1135', 33.5362979770761), ('2931', 30.76314349728561),
       ('2851', 30.692231228123816), ('616', 30.04289834028372),
       ('2470', 27.050532123110212), ('2939', 26.95259674384964),
       ('2684', 26.4023879436599), ('1145', 26.22406408378781)], 
      dtype=[('id', 'S25'), ('score', '<f8')])

## Optimisation

In [ ]:
models = [LanguageModel(indexer, lbd=i) for i in np.linspace(0,1,20)]
queries = [query(i, queriesIndexer, relevantIndexer) for i in queriesIndexer.index if i in relevantIndexer.indexFromCol]
measures = [EvalPrecisionAverage, EvalPrecisionRecall] 

EM = EvalIRModel(models, queries, measures)
results = EM.getResults()

EvalMeasure.py:19: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  np.in1d(self.irlist.scores[:i]['id'], self.irlist.query.relevants).sum()


khachian not in language!
im not in language!
rap not in language!
adaba not in language!
co-cit not in language!
pram not in language!
ecl not in language!
class-complet not in language!
topographi not in language!
tcoll not in language!
multi-target not in language!
udo not in language!
window-manag not in language!
fault-toler not in language!


In [9]:
import pickle
print(results)


[[[  5.22672494e-04   1.19966509e-03   8.61168794e-04   1.14579745e-07]
  [  9.32051766e-04   1.74332957e-03   1.33769067e-03   1.64542918e-07]
  [  2.37414816e-03   3.19871118e-03   2.78642967e-03   1.69976044e-07]
  ..., 
  [  0.00000000e+00   5.42005420e-04   2.71002710e-04   7.34424688e-08]
  [  2.15081639e-03   3.78861887e-03   2.96971763e-03   6.70599237e-07]
  [  0.00000000e+00   1.36239782e-03   6.81198910e-04   4.64031955e-07]]

 [[  5.22672494e-04   1.19966509e-03   8.61168794e-04   1.14579745e-07]
  [  9.32051766e-04   1.74332957e-03   1.33769067e-03   1.64542918e-07]
  [  2.37414816e-03   3.19871118e-03   2.78642967e-03   1.69976044e-07]
  ..., 
  [  0.00000000e+00   5.42005420e-04   2.71002710e-04   7.34424688e-08]
  [  2.15081639e-03   3.78861887e-03   2.96971763e-03   6.70599237e-07]
  [  0.00000000e+00   1.36239782e-03   6.81198910e-04   4.64031955e-07]]

 [[  5.22672494e-04   1.19966509e-03   8.61168794e-04   1.14579745e-07]
  [  9.32051766e-04   1.74332957e-03   1.337

In [22]:
len(queriesIndexer.index)
queries = [query(i, queriesIndexer, relevantIndexer) for i in queriesIndexer.index if i in relevantIndexer.indexFromCol]
print(queries)

[<Query.Query object at 0x7f32240d1510>, <Query.Query object at 0x7f32240d1410>, <Query.Query object at 0x7f32240d1110>, <Query.Query object at 0x7f32241ed850>, <Query.Query object at 0x7f32241ed8d0>, <Query.Query object at 0x7f32241ed910>, <Query.Query object at 0x7f32241ed990>, <Query.Query object at 0x7f32241ed950>, <Query.Query object at 0x7f32241ed9d0>, <Query.Query object at 0x7f32241eda10>, <Query.Query object at 0x7f32241eda50>, <Query.Query object at 0x7f32241eda90>, <Query.Query object at 0x7f32241edad0>, <Query.Query object at 0x7f32241edb50>, <Query.Query object at 0x7f32241edb10>, <Query.Query object at 0x7f32241edc10>, <Query.Query object at 0x7f32241edc50>, <Query.Query object at 0x7f32241edbd0>, <Query.Query object at 0x7f32241edc90>, <Query.Query object at 0x7f32241edb90>, <Query.Query object at 0x7f32241edd10>, <Query.Query object at 0x7f32241edd50>, <Query.Query object at 0x7f32241edcd0>, <Query.Query object at 0x7f32241edd90>, <Query.Query object at 0x7f32241eddd0>,

In [11]:
print(query(35, queriesIndexer, relevantIndexer))

ValueError: Bad Identifier